In [ ]:
import pandas as pd
import numpy as np
import vertica_python
import os
import datetime as dt
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
import _pickle as cPickle
import time

In [ ]:
os.chdir('/home/centos/notebooks/Shashank/rto_testing/')

In [ ]:
col_names = ['code', 'order_created_date', 'address_length', 'address_has_numeric',
				'dest_pincode', 'subcategory_id', 
				'bucket_id', 'user_del', 'user_del_ship', 
				'city_att_14', 'city_del_14', 'pin_att_14', 'pin_del_14',
				'courier_group', 'rto', 'Same_SUPC_Dup', 
				'Same_Subcat_Dup', 'ITR', 'del_mob_wa_sent_success', 'del_mob_wa_del', 'del_mob_wa_read', 
				'reg_mob_wa_sent_success', 'reg_mob_wa_del', 'reg_mob_wa_read', 'add1_del', 'add2_del']

select_vars = ','.join(col_names)


In [ ]:
conn_info = dict(host='10.65.0.201', port=5433, database='snapdealdwh', user='shashank.jain03',password='Sachin@200')

conn = vertica_python.connect(**conn_info)
cur = conn.cursor()
query_fetch = "select %s from analytics_logistics.ys_junfull_data_fin where shipped = 1 and subo_shipping_method_code = 'COD';" % (select_vars)
cur.execute(query_fetch)
data = cur.fetchall()
conn.close()

mydata = pd.DataFrame(data = data, columns = col_names)
mydata.to_csv('rto_data_0821.csv', index = False)


In [ ]:
mydata.tail(1)

In [ ]:
mydata = pd.read_csv('rto_data_0821.csv')
mydata = mydata.rename(columns = {'address_length': 'add_length', 'address_has_numeric': 'is_num'})
mydata['order_created_date'] = pd.to_datetime(mydata['order_created_date'], infer_datetime_format=True) 
mydata['order_created_date'] = mydata['order_created_date'].dt.date
mydata['is_num'] = np.where(mydata['is_num'] == 'Yes', 1, 0)

In [ ]:
mydata['is_num'].value_counts()

In [ ]:
mydata.head(1)

In [ ]:
mydata['add_length'] = mydata['add_length'].astype('float')

In [ ]:
# dividing data in train and test

train = mydata.loc[mydata['order_created_date'] <= dt.date(2021, 6, 15),:]
test = mydata.loc[mydata['order_created_date'] > dt.date(2021, 6, 15),:]

# creating additional variables

train = train.drop(columns = ['order_created_date', 'code'])
train = train.dropna(subset = ['subcategory_id', 'bucket_id'])
train[['subcategory_id', 'bucket_id']] = train[['subcategory_id', 'bucket_id']].astype('int64')

cols = ['user_del_ship', 'pin_att_14', 'city_att_14']
train[cols] = train[cols].replace(0,np.nan)

train.loc[train['user_del_ship'].isna(), 'user_del'] = np.nan

train['del_per'] = np.where(train['user_del_ship'] != 0, train['user_del']*100/train['user_del_ship'], np.nan)

train['pin_fasr_14'] = np.where(~train['pin_att_14'].isna(), train['pin_del_14']*100/train['pin_att_14'], np.nan)
train['city_fasr_14'] = np.where(~train['city_att_14'].isna(), train['city_del_14']*100/train['city_att_14'], np.nan)

train[['Same_SUPC_Dup', 'Same_Subcat_Dup']] = train[['Same_SUPC_Dup', 'Same_Subcat_Dup']].fillna(0)
train['ITR'] = train['ITR'].fillna(0)

train['same_supc'] = np.where(train['Same_SUPC_Dup'] > 0, 1, 0)
train['same_subcat'] = np.where(train['Same_Subcat_Dup'] > 0, 1, 0)

train['three_80'] = np.where((train['user_del_ship'] > 3) & (train['del_per'] < 20), 1, 0)
train['five_70'] = np.where((train['user_del_ship'] > 5) & (train['del_per'] < 30), 1, 0)

train.loc[train['user_del_ship'].isna(), 'three_80'] = np.nan
train.loc[train['user_del_ship'].isna(), 'five_70'] = np.nan


top_vars = ['del_per', 'pin_fasr_14',
				'add_length', 'subcategory_id',
				'city_fasr_14', 'bucket_id', 'dest_pincode', 'user_del',
				'is_num', 'same_supc', 'same_subcat', 'ITR', 'three_80', 'five_70', 'user_del_ship', 
				'del_mob_wa_sent_success', 'del_mob_wa_del', 'del_mob_wa_read', 
				'reg_mob_wa_sent_success', 'reg_mob_wa_del', 'reg_mob_wa_read']


train = train[top_vars + ['rto']]


In [ ]:

test = test.dropna(subset = ['subcategory_id', 'bucket_id'])
test[['subcategory_id', 'bucket_id']] = test[['subcategory_id', 'bucket_id']].astype('int64')

test[cols] = test[cols].replace(0,np.nan)

test.loc[test['user_del_ship'].isna(), 'user_del'] = np.nan

test['del_per'] = np.where(test['user_del_ship'] != 0, test['user_del']*100/test['user_del_ship'], np.nan)

test['pin_fasr_14'] = np.where(~test['pin_att_14'].isna(), test['pin_del_14']*100/test['pin_att_14'], np.nan)
test['city_fasr_14'] = np.where(~test['city_att_14'].isna(), test['city_del_14']*100/test['city_att_14'], np.nan)

test[['Same_SUPC_Dup', 'Same_Subcat_Dup']] = test[['Same_SUPC_Dup', 'Same_Subcat_Dup']].fillna(0)
test['ITR'] = test['ITR'].fillna(0)

test['same_supc'] = np.where(test['Same_SUPC_Dup'] > 0, 1, 0)
test['same_subcat'] = np.where(test['Same_Subcat_Dup'] > 0, 1, 0)

test['three_80'] = np.where((test['user_del_ship'] > 3) & (test['del_per'] < 20), 1, 0)
test['five_70'] = np.where((test['user_del_ship'] > 5) & (test['del_per'] < 30), 1, 0)

test.loc[test['user_del_ship'].isna(), 'three_80'] = np.nan
test.loc[test['user_del_ship'].isna(), 'five_70'] = np.nan

out_of_time = test[['order_created_date', 'code', 'rto']].copy()
test = test[top_vars + ['rto']]


In [ ]:
cat_vars = ['subcategory_id', 'bucket_id', 'dest_pincode']
train[cat_vars] = train[cat_vars].astype('str')
test[cat_vars] = test[cat_vars].astype('str')


In [ ]:
# combining long tail of categorical variables

club_vars = ['subcategory_id', 'bucket_id', 'dest_pincode']

def club_categories(x,n = 100):
    freq = x.value_counts()
    less_freq_cats = list(freq[freq < n].keys())
    x[x.isin(less_freq_cats)] = 'other'
    return(x)

train[club_vars] = train[club_vars].apply(club_categories, axis = 0)

for var in club_vars:
	train_values = train[var].unique()
	test.loc[~test[var].isin(train_values), var] = 'other'


# replacing categorical data with weight of evidence of respective categories

def iv_variable(data, indep, dep):

	sub_data = data[[indep, dep]].copy()
	event = sub_data[dep].sum()
	non_event = sub_data.shape[0] - event
	iv_data = sub_data.groupby(indep).agg(['sum', 'size']).rename(columns = {'sum': 'event', 'size': 'total'})
	iv_data.columns = iv_data.columns.droplevel(level=0)
	iv_data = iv_data.reset_index(drop = False)
	iv_data['non_event'] = iv_data['total'] - iv_data['event'] 
	iv_data['per_event'] = iv_data['event']/event
	iv_data['per_non_event'] = iv_data['non_event']/non_event
	iv_data['woe'] = np.log(((iv_data['non_event'] + 0.5)/non_event)/((iv_data['event'] + 0.5)/event)) * (iv_data['per_non_event'] - iv_data['per_event'])
	iv_data['woe'] = 100*iv_data['woe']
# 	iv_data['cat_rto'] = iv_data['event']/iv_data['total']
# 	base_rto = event/(event + non_event)
# 	iv_data['woe'] = np.where(iv_data['cat_rto'] < base_rto, iv_data['cat_rto']*(-1.0), iv_data['cat_rto'])
	iv_data = iv_data[[indep, 'woe']]
	return(iv_data)


woe_dict = {}

start = time.time()

for var in cat_vars:
	iv_var = iv_variable(train, var, 'rto')
	woe_dict[var] = dict(zip(iv_var[var], iv_var['woe']))
	train = pd.merge(train, iv_var, how = 'left', on = var).drop(columns = [var]).rename(columns = {'woe': var})
	test = pd.merge(test, iv_var, how = 'left', on = var).drop(columns = [var]).rename(columns = {'woe': var})

end = time.time()


In [ ]:
train.head(2)

In [ ]:
# imputing missing values with a fixed large negative numeric value so that it will be treated as a different category

train.fillna(-99999, inplace = True)
test.fillna(-99999, inplace = True)


In [ ]:
# dividing training into training and validation data

X, y = train.drop(columns = ['rto', 'del_mob_wa_sent_success', 'del_mob_wa_del', 'del_mob_wa_read', 'reg_mob_wa_sent_success', 'reg_mob_wa_del', 'reg_mob_wa_read']), train['rto']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size = 0.2, random_state = 163)

test_x, test_y = test.drop(columns = ['rto', 'del_mob_wa_sent_success', 'del_mob_wa_del', 'del_mob_wa_read', 'reg_mob_wa_sent_success', 'reg_mob_wa_del', 'reg_mob_wa_read']), test['rto']


In [ ]:
# training the model

xgb_clf = xgb.XGBClassifier(max_depth = 6,
                           min_child_weight = 1,
                           learning_rate = 0.1,
                           n_estimators = 200,
                           objective = 'binary:logistic',
                           gamma = 5,
                           n_jobs = 30)

 
xgb_clf.fit(train_x, train_y, eval_metric = 'error',
            eval_set = [(train_x, train_y), (valid_x, valid_y)], early_stopping_rounds = 50)



In [ ]:
# getting variable importance

var_imp = xgb_clf.get_booster().get_score(importance_type = 'total_gain')
imp_data = pd.DataFrame({'feature': list(var_imp.keys()), 'importance': list(var_imp.values())})
imp_data = imp_data.sort_values(by = 'importance', ascending = False)
imp_data.importance = imp_data.importance/np.sum(imp_data.importance)
imp_data.reset_index(drop = True, inplace = True)


In [ ]:
imp_data

In [ ]:
# getting test accuracy

predicted_values = xgb_clf.predict_proba(test_x)[:,1]
predicted_class = np.where(predicted_values > 0.5, 1, 0)
test_accuracy = np.sum(test_y == predicted_class)/len(test_y)

print('test accuracy: %.3f' % (test_accuracy))

In [ ]:
# getting test accuracy

predicted_values = xgb_clf.predict_proba(test_x)[:,1]
predicted_class = np.where(predicted_values > 0.5, 1, 0)
test_accuracy = np.sum(test_y == predicted_class)/len(test_y)

print('test accuracy: %.3f' % (test_accuracy))

In [ ]:
# getting test accuracy (new woe)

predicted_values = xgb_clf.predict_proba(test_x)[:,1]
predicted_class = np.where(predicted_values > 0.5, 1, 0)
test_accuracy = np.sum(test_y == predicted_class)/len(test_y)

print('test accuracy: %.3f' % (test_accuracy))

In [ ]:
test['rto_prob'] = predicted_values
test.to_csv('rto_probs_june21.csv', index = False)

In [ ]:
test.head(2)

In [ ]:
test = test.replace(-99999.0, np.nan)

In [ ]:
test.head(2)

In [ ]:
test.isna().sum()

In [ ]:
test.dtypes

In [ ]:
indep = 'del_per'

In [ ]:
def var_imp(indep, method = 'cut', q = 10):
    sub_data = test[[indep, 'rto', 'rto_prob']].copy()
    if method == 'cut':
        sub_data['decile'] = pd.cut(sub_data[indep], bins = 10)
    else:
        sub_data['decile'] = pd.qcut(sub_data[indep], q = q, duplicates = 'drop')
          
    sub_data['decile'] = sub_data['decile'].astype('str')

    out = sub_data.groupby('decile').agg(
        count = ('rto_prob', 'count'),
        rto_prob_mean = ('rto_prob', 'mean'),
        rto_actual = ('rto', 'mean'),   
    ).reset_index()

    out['salience'] = np.round(out['count']*100/np.sum(out['count']),2)
    out = out.rename(columns = {'decile': indep})
    out['rto_prob_mean'] = np.round(out['rto_prob_mean'],2)
    out['rto_prob_mean'] = np.round(out['rto_prob_mean'],2)
    return(out)

In [ ]:
var_imp('del_per')

In [ ]:
var_imp('pin_fasr_14')

In [ ]:
var_imp('add_length', method = 'qcut')

In [ ]:
var_imp('ITR', method = 'qcut', q = 50)

In [ ]:
out_of_time['rto_prob'] = predicted_values
# out_of_time.to_csv('rto_probs_3_80_june21.csv', index = False)
# out_of_time = pd.read_csv('rto_probs_june21.csv')

out_of_time['decile'] = pd.cut(out_of_time['rto_prob'], bins = np.arange(0,1.01, 0.05))
out_of_time['decile'] = out_of_time['decile'].astype('str').str.replace(' ', '')

out = out_of_time.groupby(['decile'])['rto'].agg(['size', 'sum']).reset_index(drop = False).rename(columns = {'size': 'count', 'sum': 'rto'})
out['rto_per'] = out['rto']/out['count']
out['salience'] = out['count']/out_of_time.shape[0]
out

In [ ]:
out_of_time['rto_prob'] = predicted_values
# out_of_time.to_csv('rto_probs_3_80_june21.csv', index = False)
# out_of_time = pd.read_csv('rto_probs_june21.csv')

out_of_time['decile'] = pd.cut(out_of_time['rto_prob'], bins = np.arange(0,1.01, 0.05))
out_of_time['decile'] = out_of_time['decile'].astype('str').str.replace(' ', '')

out = out_of_time.groupby(['decile'])['rto'].agg(['size', 'sum']).reset_index(drop = False).rename(columns = {'size': 'count', 'sum': 'rto'})
out['rto_per'] = out['rto']/out['count']
out['salience'] = out['count']/out_of_time.shape[0]
out

In [ ]:
# with new woe

out_of_time['rto_prob'] = predicted_values
# out_of_time.to_csv('rto_probs_3_80_june21.csv', index = False)
# out_of_time = pd.read_csv('rto_probs_june21.csv')

out_of_time['decile'] = pd.cut(out_of_time['rto_prob'], bins = np.arange(0,1.01, 0.05))
out_of_time['decile'] = out_of_time['decile'].astype('str').str.replace(' ', '')

out = out_of_time.groupby(['decile'])['rto'].agg(['size', 'sum']).reset_index(drop = False).rename(columns = {'size': 'count', 'sum': 'rto'})
out['rto_per'] = out['rto']/out['count']
out['salience'] = out['count']/out_of_time.shape[0]
out